# Curious Representation Learning for Embodied Intelligence
## 文献出处及作者
* 会议：ICCV 2001
* 作者<br>
Yilun Du - MIT<br>
Chuang Gan -MIT-IBM Watson AI Lab<br>
Phillip Isola - MIT
## 文献主要内容
* 目标：解决具身智能中的一个关键问题：如何让智能体在没有任何外部任务或监督标签的情况下，通过主动与环境交互来学习通用、任务无关的视觉表征。
* 核心思想：提出了一个名为 好奇表征学习（Curious Representation Learning, CRL） 的统一框架。该框架通过一个极小极大博弈 同时训练一个强化学习探索策略和一个自监督视觉表征模型：<br>
**策略目标：** 探索环境，寻找那些能让表征模型产生高预测误差的观测图像。策略的奖励被设定为表征模型的损失。<br>
**模型目标：** 努力从策略提供的“困难”图像中学习，不断降低其损失，从而变得更加强大和鲁棒。<br>
这个博弈过程形成了一个良性循环：策略为了获得高奖励，会不断寻找模型不熟悉的、新颖的图像，从而为模型提供了持续且有挑战性的数据；而模型通过在这些数据上学习，其表征能力不断增强，又迫使策略去探索更具挑战性的区域。
## 关键贡献：
1. 提出了CRL框架，将好奇心探索与自监督表征学习有机结合
2. 证明了所学表征能有效提升多种下游具身任务（如导航）的性能，并且在迁移到下游任务时必须冻结视觉编码器以防止性能下降。
3. 展示了尽管仅在仿真环境中训练，所学表征能够迁移到真实世界图像的理解上，并产生可解释的结果。
## 好奇表征学习CRL
* 目标：展示了尽管仅在仿真环境中训练，所学表征能够迁移到真实世界图像的理解上，并产生可解释的结果。
* 代码：https://yilundu.github.io/crl/
### 对比表示学习
* 目标：学习到一个视觉编码器$M_\phi$，它能够将图像映射到一个特征空间，使得同一张图像的不同增强版本（“正样本对”）在特征空间中非常接近，而不同图像的增强版本（“负样本对”）则相距甚远，这样学习到的表征不依赖于人工标签，能够捕捉图像的本质特征
* 关键组件：<br>
**表征模型$M_\phi$：** ResNet50，从图像中提取特征<br>
**投影头$g_\psi$：** 一个2层的MLP，将ResNet提取的特征映射到一个更适合做对比学习的低维空间。<br>
**一组数据增强$\tau$：** 定义了一系列随机的图像变换，包括：水平翻转、随机缩放裁剪、颜色饱和度变化。这些增强用于创造“正样本对”。
* 工作流程：<br>
**1.** 对于一个包含N张图像的批次${x_k}$<br>
**2.** 对批次中的每一张图像$x_i$，从$\tau$中独立采样两次，得到两个不同的增强版本$\tilde{x}_i^1$和$\tilde{x}_i^2$。构成N个正样本对。<br>
**3.** 将2N张增强后的图片依次通过$M_\phi$和$g_\psi$。<br>
**4.** 对投影头的输出进行L2归一化，得到最终的潜向量$z$。
* 损失函数：<br>
使用**InfoNCE**作为损失函数：<br>
$$L_contrast = -\frac{1}{N} \sum^N_{i=1}log\frac{exp(sim(\tilde{z}^1_i,\tilde{z}^2_i)/\tau)}{\sum^N_{j=1}\sum^N_{k=1}exp(sim(\tilde{z}^1_j,\tilde{z}^2_k)/\tau))}$$<br>
分子：衡量正样本对$(\tilde{z}^1_i,\tilde{z}^2_i)$的相似度，鼓励它越大越好<br>
分母：衡量一个正样本对与批次中所有其他样本的相似度<br>
参数$\tau$：用于调节分布的尖锐程度
### 基于内在特征的表征学习（核心创新点）
1. 从静态数据到交互环境的范式转变<br>
**传统：** 模型被动的从固定数据集$p_data$中学习。目标是$min_\phi E_{x\sim p_{data}}[L_rep]$<br>
**CRL：** 数据由策略$\pi_\theta$主动选择，策略的目标是最大化累积奖励：$max_\theta E_{x\sim \pi_\theta}[\sum r_t]$
2. 好奇心奖励<br>
CRL将表征模型的损失函数直接作为策略的奖励<br>
在每一个时间步t，策略获得的奖励$r_t=L_{rep}(M_\phi,x_t)$<br>，因此，策略的优化目标为：
$$\underset{\theta}{max}E_{x\sim \pi_\theta}[\sum^T_{t=0}L_{rep}(M_\phi,x)]$$
3. 极小极大博弈<br>
将表征模型的目标和策略的目标集合起来，就得到了CRL的总体目标函数：<br>
$$\underset{\theta}{max}\underset{\phi}{min}E_{x\sim \pi_\theta}[\sum^T_{t=0}L_{rep}(M_\phi,x)]$$<br>
**策略$\pi_\theta$：** 试图找到让模型$M_\phi$损失最大的数据<br>
**模型$M_\phi$：** 努力从策略提供的困难数据中学习，最小化自己的损失<br>
这个博弈促使策略不断探索新区域，同时模型也变得更强，鲁棒性更高
4. 和以往好奇心方法的差异
**以往的方法：** 需要手动设计一个预测任务定义“新奇性”
**CRL：** 提供了一个通用框架，任何先进的自监督表征学习算法都可以直接插入作为好奇心目标
### 模型与策略优化
1. 奖励函数的修正<br>
**问题：** 直接使用完整的对比损失$L_contrast$作为奖励，策略会发现一个**作弊方法**：让智能体静止不动。因为所有观测图像都相同，负样本对的相似度会极高，导致分母巨大，从而使损失值$L_contrast$变得很大，策略不探索就能获得高奖励<br>
**解决方案：** 将奖励重新定义为只包含对比损失分子部分的负数：$r_t=-sim(x^1_t,x^2_t)$，再加上一个常数，确保奖励非负<br>
**有效性：** 这个奖励只关心正本对之间的一致性。如果策略静止不动，所有正样本对的相似度会非常高，导致奖励变低。因此，为了获得高奖励，策略必须去寻找那些即使经过数据增强，其表征也难以保持一致的图像，这就促进了视觉上多样和新奇的观测
2. 训练稳定化技巧
**奖励归一化：** 对奖励进行归一化，防止奖励尺度的剧烈变化
**网络架构调整：** 将ResNet中的批归一化层替换为组归一化层。因为批归一化再RL的在线学习环境中效果不佳。
3. 优化算法
**近端优化策略(PPO)：** 使用PPO训练$\pi_\theta$
